# CSILSP-NCS

In [1]:
import gurobipy as gb
from CSILSPGenerator import GeneratorTW_NCS

#### Decision variables:

$$
X_t = \begin{cases}
\text{ quantity of product to be produced/supplied in period t }
\end{cases}
$$

$$
I_t = \begin{cases}
\text{ inventory level at the end of period t }
\end{cases}
$$

$$
Y_t = \begin{cases}
1 \text{ if } X_t > 0\\
0 \text{ otherwise }
\end{cases}
$$

#### Parameters:

$$
T = \begin{cases}
\text{ number of periods in the horizon }
\end{cases}
$$

$$
d_t = \begin{cases}
\text{ Customer aggregate demand that must } \\ \text{ be delivered at the end of period t }
\end{cases}
$$

$$
h_t = \begin{cases}
\text{ Holding cost per unit of product in stock at the end of period t }
\end{cases}
$$

$$
p_t = \begin{cases}
\text{ Unit production/supplying cost in period t }
\end{cases}
$$

$$
s_t = \begin{cases}
\text{ Fixed setup cost in period t }
\end{cases}
$$

$$
C_t = \begin{cases}
\text{ Production/Supply capacity at period t }
\end{cases}
$$

#### CSILSP-NCS

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T} (h_tI_t + p_tX_t + s_tY_t) &\\
\text{s.t.} \;\;\;\;\;&\\
I_t &= I_{t-1} + X_t - d_t \;\; , & \forall t \;\;\\
X_t &\leq \sum_{k=t}^{T} d_kY_t \;\; , & \forall t\;\;\\
X_t,I_t &\geq 0 \;\; , & \forall t\;\;\\
X_t & \leq C_tY_t \;\; , & \forall t\;\;\\
\sum_{k=1}^{t}X_k &\leq \sum_{k=1}^{t}\sum_{j=k}^{T}d_{k,j} \;\; , & \forall t\;\;\\
Y_t &\in \{0,1\} \;\; , & \forall t \;\;
\end{alignat}
$$

In [2]:
# generation parameters

if __name__ == '__main__':
    T = 8
    l_b = 10
    u_b = 20

In [3]:
# generate instance

if __name__ == '__main__' or __name__ == '__autoexec__':
    instance = GeneratorTW_NCS(T)
    instance.generate(a=l_b,b=u_b,dw_a=l_b,dw_b=u_b)

In [4]:
if __name__ == '__main__':
    print(instance.printData())

      d      C   h   p   s
T                         
0   0.0  200.0  12  15  14
1  16.0  206.0  20  12  20
2  14.0  200.0  16  17  12
3  18.0  205.0  16  17  15
4  10.0  202.0  15  17  20
5  19.0  205.0  20  19  11
6  15.0  198.0  14  13  14
7  96.0  204.0  13  19  15


In [5]:
if __name__ == '__main__':
    print(instance.get_dw())

{(0, 0): 0, (0, 1): 16, (0, 2): 14, (0, 3): 18, (0, 4): 10, (0, 5): 19, (0, 6): 15, (0, 7): 13, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 13, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (2, 6): 0, (2, 7): 10, (3, 3): 0, (3, 4): 0, (3, 5): 0, (3, 6): 0, (3, 7): 16, (4, 4): 0, (4, 5): 0, (4, 6): 0, (4, 7): 10, (5, 5): 0, (5, 6): 0, (5, 7): 14, (6, 6): 0, (6, 7): 20, (7, 7): 0}


In [6]:
# variable bindings

T = instance.get_T()
d_w = instance.get_dw()
d = instance.get_d()
C = instance.get_C()
h = instance.get_h()
p = instance.get_p()
s = instance.get_s()

In [7]:
# model

csilsp_ncs = gb.Model()

Academic license - for non-commercial use only


In [8]:
# decision variables

X = csilsp_ncs.addVars(T,vtype=gb.GRB.CONTINUOUS,lb=0.0, name='X')

I = csilsp_ncs.addVars(T,vtype=gb.GRB.CONTINUOUS,lb=0.0, name='I')

Y = csilsp_ncs.addVars(T,vtype=gb.GRB.BINARY,name='Y')

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T} (h_tI_t + p_tX_t + s_tY_t) &
\end{alignat}
$$

In [9]:
# objective function

csilsp_ncs.setObjective(I.prod(h) + X.prod(p) + Y.prod(s),gb.GRB.MINIMIZE)

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

$$
\begin{alignat}{3}
I_t &= I_{t-1} + X_t - d_t \;\; , & \forall t
\end{alignat}
$$

In [10]:
# constraints

csilsp_ncs.addConstrs( ( I[t] == I[t-1] + X[t] - d[t] for t in range(1,T-1) ),name='constrI')

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

In [11]:
# Assumption: starting and ending inventories are null

csilsp_ncs.addConstr(I[0] == 0,name='startInventoryNull')
csilsp_ncs.addConstr(I[T-1] == 0,name='endInventoryNull')

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

$$
\begin{alignat}{3}
X_t &\leq \sum_{k=t}^{T} d_kY_t \;\; , & \forall t
\end{alignat}
$$

In [12]:
csilsp_ncs.addConstrs( ( X[t] <= (gb.quicksum(d[k] for k in range(t,T)))*Y[t] for t in range(T) ),name='constX')

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

Utilizzare vincolo (6) o vincolo (7):

$$
\begin{alignat}{3}
X_t & \leq C_t \;\; , & \forall t \;\; (6)
\end{alignat}
$$

In [13]:
#csilsp_ncs.addConstrs( ( X[t] <= C[t] for t in range(T) ),name='constrC')

#csilsp_ncs.update()
#csilsp_ncs.write('csilsp_ncs.lp')

$$
\begin{alignat}{3}
X_t & \leq C_tY_t \;\; , & \forall t \;\; (7)
\end{alignat}
$$

In [14]:
csilsp_ncs.addConstrs( ( X[t] <= C[t]*Y[t] for t in range(T) ),name='constrC')

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

$$
\begin{alignat}{3}
\sum_{k=1}^{t}X_k &\leq \sum_{k=1}^{t}\sum_{j=k}^{T}d_{k,j} \;\; , & \forall t\;\;
\end{alignat}
$$

In [15]:
csilsp_ncs.addConstrs( ( (gb.quicksum(X[k] for k in range(t+1))) <= \
                         (gb.quicksum(d_w[k,j] for k in range(t+1) for j in range(k,T))) \
                         for t in range(T) ),name='constrTimeWindows')

csilsp_ncs.update()
csilsp_ncs.write('csilsp_ncs.lp')

In [16]:
# solve model

csilsp_ncs.optimize()

Optimize a model with 32 rows, 24 columns and 88 nonzeros
Variable types: 16 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 21 rows and 10 columns
Presolve time: 0.00s
Presolved: 11 rows, 14 columns, 31 nonzeros
Variable types: 9 continuous, 5 integer (5 binary)

Root relaxation: objective 1.507628e+03, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1507.62838    0    4          - 1507.62838      -     -    0s
H    0     0                    1554.0000000 1507.62838  2.98%     -    0s
     0     0     cutoff    0      1554.00000 1554.00000  0.00%     -    0s

Explored 1 nodes (14 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 155

In [17]:
if __name__ == '__main__':
    print("Soluzione ottima:",csilsp_ncs.ObjVal)

Soluzione ottima: 1554.0


In [18]:
X

{0: <gurobi.Var X[0] (value 0.0)>,
 1: <gurobi.Var X[1] (value 16.0)>,
 2: <gurobi.Var X[2] (value 14.0)>,
 3: <gurobi.Var X[3] (value 18.0)>,
 4: <gurobi.Var X[4] (value 10.0)>,
 5: <gurobi.Var X[5] (value 19.0)>,
 6: <gurobi.Var X[6] (value 15.0)>,
 7: <gurobi.Var X[7] (value 0.0)>}

In [19]:
Y

{0: <gurobi.Var Y[0] (value 0.0)>,
 1: <gurobi.Var Y[1] (value 1.0)>,
 2: <gurobi.Var Y[2] (value 1.0)>,
 3: <gurobi.Var Y[3] (value 1.0)>,
 4: <gurobi.Var Y[4] (value 1.0)>,
 5: <gurobi.Var Y[5] (value 1.0)>,
 6: <gurobi.Var Y[6] (value 1.0)>,
 7: <gurobi.Var Y[7] (value 0.0)>}

In [20]:
I

{0: <gurobi.Var I[0] (value 0.0)>,
 1: <gurobi.Var I[1] (value 0.0)>,
 2: <gurobi.Var I[2] (value 0.0)>,
 3: <gurobi.Var I[3] (value 0.0)>,
 4: <gurobi.Var I[4] (value 0.0)>,
 5: <gurobi.Var I[5] (value 0.0)>,
 6: <gurobi.Var I[6] (value 0.0)>,
 7: <gurobi.Var I[7] (value 0.0)>}

In [21]:
# execution time
if __name__ == '__main__':
    print("Optimize time: %.5g sec" % csilsp_ncs.Runtime)

Optimize time: 0.063972 sec


In [22]:
# Write results to file

results = "----------------------------------------------------------------" + "\n"
results += " Parameter T = %d " % (T) + "\n"
results += " Parameters d,h,p,s,C random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Time window demands (d_w) random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Execution time = %.5g sec " % csilsp_ncs.Runtime + "\n"
results += "----------------------------------------------------------------" + "\n\n\n"

In [23]:
with open("csilsp_ncs_results.txt", "a") as write_file:
    write_file.write(results)